In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

In [ ]:
driver = webdriver.Chrome();
url="https://www.bseindia.com/markets/equity/EQReports/MarketWatch.html?index_code=16"
driver.get(url)
#wait until the submit button gets rendered
submit_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-default")))
ActionChains(driver).click(submit_button).perform()
#wait until the result after clicking submit_button renders
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tdcolumn.text-left")))
page_contents=driver.page_source

In [ ]:
if "ZOMATO" not in page_contents:
    raise Exception("Page after submit not loaded")

Things that I want to extract - 
open, high, low, name

In [ ]:
soup = BeautifulSoup(page_contents, "html.parser")
bse_data = {
    "company_codes": [],
    "company_names":[],
    "opening_price":[],
    "high_price":[],
    "low_price":[],
}
rows = soup.find_all("tr", attrs={"class": "ng-scope"})

#for loop to populate bse_data
for row in rows:
    valid_tag_count=0
    for child in row.children:
        if(valid_tag_count==6):
            break
        if(child.name is not None):
            if(valid_tag_count==0):
                bse_data["company_codes"].append(child.text.strip())
            elif(valid_tag_count==1):
                bse_data["company_names"].append(child.text.strip())
            elif(valid_tag_count==3):
                bse_data["opening_price"].append(child.text.strip())
            elif(valid_tag_count==4):
                bse_data["high_price"].append(child.text.strip())
            elif(valid_tag_count==5):
                bse_data["low_price"].append(child.text.strip())
            valid_tag_count+=1

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(bse_data)
print(df)

In [ ]:
#close driver and export bse_data
driver.quit()
%store bse_data

In [ ]:
from selenium.webdriver.support.select import Select
import time
import pandas as pd
driver = webdriver.Chrome();
one_year_data_bse = {
    "date": [],
    "open": [],
    "high": [],
    "low": [],
    "close": []
}
one_year_url="https://www.bseindia.com/indices/IndexArchiveData.html"
driver.get(one_year_url)

select_index=WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, "ddlIndex")))
time.sleep(2)
select=Select(select_index);
select.select_by_visible_text('BSE SENSEX')
select_index=WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, "ddlPeriodtype")))
select=Select(select_index);
select.select_by_visible_text('Daily')

end_dates=["30", "31", "30", "31", "31", "30", "31", "30", "31", "31", "28", "31"]
month=["04", "05", "06", "07", "08", "09", "10", "11", "12", "01", "02", "03"]
year=["2024", "2025"]

y_ind=0
for i in range(len(end_dates)):
    if(month[i]=="01"):
        y_ind=1
    from_element=WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, "txtFromDt")))
    to_element=WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, "txtToDt")))
    start_date="01/"+month[i]+"/"+year[y_ind]
    driver.execute_script("arguments[0].value = arguments[1];", from_element, start_date)
    driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", from_element)
    end_date=end_dates[i]+"/"+month[i]+"/"+year[y_ind]
    driver.execute_script("arguments[0].value = arguments[1];", to_element, end_date)
    driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", to_element)
    submit_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-default")))
    driver.execute_script("arguments[0].click();", submit_button)
    
    table=WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table tbody table tbody")))
    time.sleep(5)
    rows = table.find_elements(By.CSS_SELECTOR, "tr")
    for index in range(len(rows)):
        row = table.find_elements(By.CSS_SELECTOR, "tr")
        cols=row[index].find_elements(By.TAG_NAME, "td")
        for j in range(len(cols)):
            if(j==0):
                one_year_data_bse["date"].append(cols[j].text.strip())
            if(j==1):
                one_year_data_bse["open"].append(cols[j].text.strip())
            if(j==2):
                one_year_data_bse["high"].append(cols[j].text.strip())
            if(j==3):
                one_year_data_bse["low"].append(cols[j].text.strip())
            if(j==4):
                one_year_data_bse["close"].append(cols[j].text.strip())
df=pd.DataFrame.from_dict(one_year_data_bse)
df.head()

In [ ]:
driver.quit()
%store one_year_data_bse